<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" style="padding-right:10px;width:140px;float:left"></td>
<h2 style="white-space: nowrap">Image Processing Laboratory Notebooks</h2>
<hr style="clear:both">
<p style="font-size:0.85em; margin:2px; text-align:justify">
This Juypter notebook is part of a series of computer laboratories which are designed
to teach image-processing programming; they are running on the EPFL's Noto server. They are the practical complement of the theoretical lectures of the EPFL's Master course <b>Image Processing I</b> 
(<a href="https://moodle.epfl.ch/course/view.php?id=522">MICRO-511</a>) taught by Prof. M. Unser and Prof. D. Van de Ville.
</p>
<p style="font-size:0.85em; margin:2px; text-align:justify">
The project is funded by the Center for Digital Education and the School of Engineering. It is owned by the <a href="http://bigwww.epfl.ch/">Biomedical Imaging Group</a>. 
The distribution or the reproduction of the notebook is strictly prohibited without the written consent of the authors.  &copy; EPFL 2022.
</p>
<p style="font-size:0.85em; margin:0px"><b>Authors</b>: 
    <a href="mailto:pol.delaguilapla@epfl.ch">Pol del Aguila Pla</a>, 
    <a href="mailto:kay.lachler@epfl.ch">Kay Lächler</a>,
    <a href="mailto:alejandro.nogueronaramburu@epfl.ch">Alejandro Noguerón Arámburu</a>, and
    <a href="mailto:daniel.sage@epfl.ch">Daniel Sage</a>.
</p>
<hr style="clear:both">
<h1>Lab 3: Morphology</h1>
<div style="background-color:#F0F0F0;padding:4px">
    <p style="margin:4px;"><b>Released</b>: Thursday December 8, 2022</p>
    <p style="margin:4px;"><b>Submission</b>: <span style="color:red">Friday December 23, 2022</span> (before 23:59) on <a href="https://moodle.epfl.ch/course/view.php?id=522">Moodle</a></p>
    <p style="margin:4px;"><b>Grade weight</b>: Lab 3 (17 points), 10% of the overall grade</p>
    <p style="margin:4px;"><b>Help session</b>: Thursday December 22, 10h00-12h00, CO2 </p>    
    <p style="margin:4px;"><b>Related lectures</b>: Chapter 4</p> 
</div>

### Student Name: 
### SCIPER: 

Double-click on this cell and fill your name and SCIPER number. Then, run the cell below to verify your identity in Noto and set the seed for random results.

In [ ]:
import getpass
# This line recovers your camipro number to mark the images with your ID
uid = int(getpass.getuser().split('-')[2]) if len(getpass.getuser().split('-')) > 2 else ord(getpass.getuser()[0])
print(f'SCIPER: {uid}')

## Imports
In this first cell we import the required Python libraries:
* [`matplotlib.pyplot`](https://matplotlib.org) to display images,
* [`ipywidgets`](https://ipywidgets.readthedocs.io/en/latest/) to make the image display interactive,
* [`numpy`](https://numpy.org/doc/stable/reference/index.html) for mathematical operations on arrays, and
* [`cv2`](https://docs.opencv.org/master/) for image processing in Python.

We will then load the `ImageViewer` class. For more information on it, you can either see the complete documentation [here](https://github.com/Biomedical-Imaging-Group/interactive-kit/wiki/Image-Viewer), run the Python command `help(viewer)` after loading the class, or refer to [Lab 0: Introduction](../0_Introductory_lab/Introductory.ipynb)).

Finally, we load the images you will use in the exercise to test your algorithms.

In [ ]:
# Configure plotting as dynamic
%matplotlib widget

# Import standard required packages for this exercise
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import cv2 as cv

from interactive_kit import imviewer as viewer

# Load images to be used in this lab
# We import module io to import tif images as slices
from skimage import io
# The following cell loads the image stack mouse-tracking
mouse_tracking = io.imread('images/mouse-tracking.tif') 
notes = cv.imread('images/notes.tif', cv.IMREAD_UNCHANGED)
test_contact = cv.imread('images/test-contact.tif', cv.IMREAD_UNCHANGED)
test_scratch = cv.imread('images/test-scratch.tif', cv.IMREAD_UNCHANGED)

Now run the next cell to re-declare the function `disc` from the first part of the lab, that creates a circular structuring element.

In [ ]:
# Function that generates a disc structuring element in python
def disc(n):
    # Define the function of a circle as a lambda function
    circle_func = lambda i, j: ((i - n//2)**2 + (j - n//2)**2) <= (n//2)**2
    # Set all elements of the array that are inside the circle of diamater n to 1 - np.uint8 to match the type used by OpenCV for structuring elements
    output = np.fromfunction(circle_func, shape=(n,n)).astype(np.uint8)
    # Return the structuring element
    return output

# Morphology Applications (5 points)
In the previous part of the lab, you looked in detail at the pixel-wise implementation of morphological operations. In this second part, we will look at the applications of these operations, and how they are used in image processing workflows using OpenCV.

## <a id="ToC_2_Morphology"></a>Table of contents

1. [Direct applications](#1.-Direct-applications-(2-points)) (**2 points**)
    1. [Disconnecting round objects](#1.A.-Disconnecting-round-objects)
    2. [Detecting horizontal lines](#1.B.-Detecting-horizontal-lines)
2. [Combining morphological filters](#2.-Combining-morphological-filters-(1-point)) (**1 point**)
3. [Mouse tracking](#3.-Mouse-tracking-(2-points)) (**2 points**)
    1. [Mouse extraction](#3.A.-Mouse-extraction)
    2. [Implement the tracking algorithm](#3.B.-Implement-the-tracking-algorithm)
4. [Cartoonize your picture!](#4.-Cartoonize-your-picture!)

# 1. Direct applications (2 points)
[Back to table of contents](#ToC_2_Morphology)

Try to solve the following problems as best as you can. Use **only one** of the morphological filters presented in [the first part of the lab](./1_Morphology_Implementations.ipynb#3.-Morphological-filters-(9-points)), followed by a threshold operation. Make sure to select an appropriate threshold value to get a binary image ($x \in \{0, 255\}$).

<div class="alert alert-warning">
<b>Technical Note</b>: A threshold operation on <code>img</code>, given a threshold <code>thresh</code>, can be performed using <a href="https://numpy.org/doc/stable/reference/generated/numpy.where.html"><code>np.where</code></a> like this:<br> 
<code>img_binary = np.where(img > thresh, 255, 0).astype(np.uint8)</code>, which sets all pixel values of <code>img</code> bigger than <code>thresh</code> to $255$ and the others to $0$.
</div>

<div class="alert alert-info">
    
<b>Note:</b> 
    <ul><li>This is a python-only notebook, and the functions you defined before in JavaScript are not declared here, so <b>use the OpenCV operators</b>. </li>
        <li>Throughout this notebook, whenever necessary, the boundary condition should be specified as <b>reflective<b>.</li>
    </ul>
        </div>

<div class='alert alert-danger'>
    Using <b>only one</b> morphological operator means that if you want to do an opening, for example, use the <code>cv.morphologyEx</code> function with the correct parameters instead of performing the opening manually using <code>cv.erode</code> and <code>cv.dilate</code>.
</div>

## 1.A. Disconnecting round objects
[Back to table of contents](#ToC_2_Morphology)

In the image `test_contact`, **for 1 point**, disconnect the roundish white objects while keeping at least 1 white pixel per roundish object.

<table><tr>
<td>
  <p align="center" style="padding: 0px">
    <img alt="test-contact showcase" src="images/test-contact_showcase.png" width="500"><br>
  <em style="color: grey">Using only one morphological operator, you can separate all the roundish objects, as shown above.</em>
  </p>
</td>
</tr></table>

Run the next cell to visualize the image you will be working with in this exercise. Then insert your code into the function `disconnect`, that takes as parameters the original image `img` and the size `n` of the structuring element to use.

<div class="alert alert-info">
<b>Hints</b>: <ul><li>In this exercise you don't need to explicitly threshold anything because the original image is already binary.</li><li>Remember to try the different parameters of a morphological operation! For example, you can change the operator, the size and the type of a structural element.</li></ul>
</div>

In [ ]:
img_vis = viewer(test_contact)

In [ ]:
# Function that performs a single morphological filter with some structuring element of size n
def disconnect(img, n):
    # Initialize output
    output = np.zeros(img.shape)
    
    # YOUR CODE HERE

    return output

Run the next cell to check that the result only consists of binary values **(object = 255, background = 0)** using [`np.unique()`](https://numpy.org/doc/stable/reference/generated/numpy.unique.html).

In [ ]:
# Check that the output is binary
check_bin = disconnect(test_contact, 25)
if not np.all(np.unique(check_bin)==[0,255]):
    print('WARNING!\nThe output is not binary with values {0, 255}.')
else:
    print("Good, the output is binary with values {0,255}.")

To make it easy for you to find the right size of the structuring element, we will add an interactive slider to the image display using the `ImageViewer` class. Run the next cell to use the interactive widget.
<div class="alert alert-info">

<b>Note:</b> To use the slider, click the the button <code>Extra Widgets</code>. Then you can adjust the size of the structuring element with the slider and click the button <code>Disconnect</code> to apply the <code>disconnect()</code> function with the currently selected size on the original image.
</div>

In [ ]:
# Instantiate the size slider
size_slider = widgets.IntSlider(value=25, min=0, max=50, step=1, description='n')
# Instantiate the diconnect button
button = widgets.Button(description='Disconnect')

# Define the callback function of the button
def button_callback(image):
    # run the disconnect function on the image with the size indicated by the slider
    output = disconnect(image, n=size_slider.value)
    return output

# Display the image with the extra slider functionality
plt.close('all')
test_contact_display = viewer(test_contact, title="Disconnect test_contact", new_widgets=[size_slider, button], callbacks=[button_callback], widgets=True)

In the following cell, assign `n` with the size of the structuring element you think works best for this task. This number may be different depending on the structuring element you used in the `disconnect()` function.

In [ ]:
# Assign the size of the structuring element
n = None
# YOUR CODE HERE

In [ ]:
# Perform a sanity check on n
if not 0 < n < 50: 
    print('WARNING!\nThe chosen size doesn\'t really make sense.')

## 1.B. Detecting horizontal lines
[Back to table of contents](#ToC_2_Morphology)

In the image `test_scratch`, **for 1 point**, detect the horizontal white lines of thickness of 1 or more pixels.

<table><tr>
<td>
  <p align="center" style="padding: 0px">
    <img alt="test-scratch showcase" src="images/test-scratch_showcase.png" width="500"><br>
<em style="color: grey">With <b>only one</b> morphological operation and a binarization, you can extract the second image from the first one.</em>
  </p>
</td>
</tr></table>

Run the next cell to visualize the image you will be working with in this exercise, and then extract the horizontal lines in the cell after that.

<div class="alert alert-info">
<b>Note:</b> Insert your code into the function <code>detect_hlines(img, n, threshold)</code>, that takes as parameters the original image <code>img</code>, the size <code>n</code> of the structuring element to use and the binary threshold <code>t</code>.
</div>

In [ ]:
plt.close('all')
img_vis = viewer(test_scratch)

In [ ]:
# Function that performs a single morphological filter with some structuring element of size n followed by thresholding
def detect_hlines(img, n, threshold):
    # Initialize output
    output = np.zeros(img.shape)
    
    # YOUR CODE HERE

    return output

Run the next cell to check that the result only consists of binary values **(object = 255, background = 0)** using [`np.unique()`](https://numpy.org/doc/stable/reference/generated/numpy.unique.html).

In [ ]:
# You can enter any values that work for you for the size and threshold
n = 25
threshold = 125

# Check that the output is binary
check_bin = detect_hlines(test_scratch, n, threshold)
if not (len(np.unique(check_bin)) == 2 and np.allclose(np.unique(check_bin), [0, 255])):
    print('WARNING!\nThe output is not binary with values {0, 255}. Your output consists of the following values:\n' + 
          str(np.unique(check_bin)))
else:
    print("Good, the output is binary with values {0, 255}.")

Again, we will make it easier for you to select the appropriate size and threshold values by adding an extra widget to the image display. Run the next cell to use the interactive widget.

In [ ]:
# Instantiate the size slider
size_slider = widgets.IntSlider(value=25, min=0, max=50, step=1, description='n')
# Instantiate the threshold slider
thresh_slider = widgets.IntSlider(value=125, min=0, max=255, step=1, description='threshold')
# Instantiate the diconnect button
button = widgets.Button(description = 'Detect H-lines')

# Define the callback function of the button
def button_callback(image):
    # Run the disconnect function on the image with the size indicated by the slider
    output = detect_hlines(image, n=size_slider.value, threshold=thresh_slider.value)
    return output

# Display the image with the extra slider functionality
plt.close('all')
test_scratch_display = viewer(test_scratch, title="Detect horizontal lines", new_widgets=[size_slider, thresh_slider, button], 
                              callbacks=[button_callback], widgets=True)

In the following cell, assign `n` with the size of the structuring element and `threshold` with the threshold you think works best for this task.

In [ ]:
# Assign your values here
n = None
threshold = None

# YOUR CODE HERE

In [ ]:
# Perform a sanity check on n
if not 0 < n < 50: 
    print('The chosen size doesn\'t really make sense.')

In [ ]:
# Perform a sanity check on threshold
if not 0 < threshold < 255: 
    print('The chosen threshold does not really make sense.')

# 2. Combining morphological filters (1 point)
[Back to table of contents](#ToC_2_Morphology)

**For 1 point**, using **one or several** of the morphological filters and threshold operations, detect only the (filled) black note heads of the image <code>notes</code> and replace them by small (e.g. $7 \times 7$) <b>black squares</b>. Don't get desperate if you don't manage to create exact $7 \times 7$ squares, the most important thing is that all 50 black notes can be well distinguished from each other and that only the note heads are visible.

<table><tr>
<td>
  <p align="center" style="padding: 0px">
    <img alt="extracted_note_showcase" src="images/extracted_notes_showcase.png" width="500"><br>
  </p>
</td>
</tr></table>

Insert your code into the function `extract_notes(img)`.

<div class="alert alert-success">
<b>Hints:</b> <ul><li>Because you can use as many operators as you like with different structuring elements of multiple sizes, it would be too complicated to generate interactive sliders for all of them. That means you need to <b>hard-code the sizes of the structuring elements and the threshold value directly into the code</b> and change them by hand to find a combination that works for you.</li><li>Design your workflow thinking in terms of simple tasks! For example, what kind of features do you want to keep or get rid of?</li></ul>
</div>

In [ ]:
def extract_notes(img):
    output = np.zeros(img.shape)
    
    # YOUR CODE HERE
    
    return output

# Display the result
plt.close('all')
view = viewer([notes, extract_notes(notes)], title=['notes', 'Extracted note heads'], subplots=(1,2))

Run the next cell to check that your image is binary.

In [ ]:
# Check that the output is binary
check_bin = extract_notes(notes)
if not (len(np.unique(check_bin)) == 2 and np.allclose(np.unique(check_bin), [0, 255])):
    print('WARNING!\nThe output is not binary with values {0, 255}. Your output consists of the following values:\n' + 
          str(np.unique(check_bin)))
else:
    print("Good, the output is binary with values {0, 255}.")

Now run the following cell to check that your function extracts the 50 note heads.

In [ ]:
# Apply the function
extracted_notes = extract_notes(notes)
# Invert the resulting image because the note heads are black
inverted_img = (np.ones(extracted_notes.shape) * np.max(extracted_notes) - extracted_notes).astype(np.uint8)
# Count the number of note heads using cv.findContours
num_notes = len(cv.findContours(inverted_img.astype(np.uint8), cv.RETR_LIST, cv.CHAIN_APPROX_NONE)[0]) # 50
# Check the count
if num_notes != 50:
    print(f'WARNING!\nYour function extracted {num_notes} instead of 50 note heads. ' +
          'Look at your code again and try to extract the 50 black note heads.')
else:
    print('Well done, your function extracted the 50 note heads!')

# 3. Mouse tracking (2 points)
[Back to table of contents](#ToC_2_Morphology)

<table><tr>
<td>
  <p align="center" style="padding: 0px">
    <img alt="mouse_tracking_showcase" src="images/mouse_tracking_showcase.png" width="300"><br>
  </p>
</td>
</tr></table>

In this exercise we want to build a fully functioning tracking algorithm that can track the path of a mouse in an experimental setup. The complete algorithm will rely on the following functions:

* `extract_mouse` : Extracts the mouse body from the rest of the image, using one morphological operator.
* `cog` : Calculates the center of gravity of the extracted mouse body. (Already implemented)
* `show_cog` : Draws a red cross at the location of the center of gravity. (Already implemented)
* `draw_line` : Draws a line between two points on top of an image. (Already implemented)

The final function `track_mouse` will combine all these functions to draw the path of the mouse over multiple image frames. Your job will be to complete the functions `extract_mouse` and `track_mouse`. The other functions have already been implemented for you.

First of all, run the next cell to display the image stack `mouse_tracking`, which consists of 25 imges of a mouse moving around in an experimental setup. Running this cell will also define the three functions `cog`, `show_cog` and `draw_line`, which we will use later.

In [ ]:
# Calculates the center of gravity
def cog(img):
    # Create coordinate system
    X, Y = np.meshgrid(range(img.shape[1]), range(img.shape[0]))
    # Normalize image to min = 0
    tmp = img - np.min(img)
    # Check if we have a zero image to avoid division by zero
    if np.sum(tmp) == 0:
        return img.shape[1] // 2, img.shape[0] // 2
    # Calculate center of gravity
    x = np.sum(X * tmp) / np.sum(tmp)
    y = np.sum(Y * tmp) / np.sum(tmp)
    # Round to get pixel values
    return (np.round(x).astype(int), np.round(y).astype(int))

# Function that draws a red cross at the center of gravity
def show_cog(img, cog_pos=None, mask=None):
    # Check mask
    mask = mask if mask is not None else img
    # Get center of gravity
    cog_x, cog_y = cog_pos if cog_pos is not None else cog(mask)
    # Create RGB output image from input image
    tmp = (img - np.min(img)) / (np.max(img) - np.min(img)) * 255
    output = cv.cvtColor(tmp.astype(np.uint8), cv.COLOR_GRAY2RGB)
    # Add red 9x9 cross at center of gravity
    output[cog_y-4:cog_y+5, cog_x] = np.array([255, 0, 0]) # Red
    output[cog_y, cog_x-4:cog_x+5] = np.array([255, 0, 0]) # Red
    return output

# Function that draws a line from (x0, y0) to (x1, y1)
def draw_line(img, color, x0, y0, x1, y1):
    output = np.copy(img)
    # Get the maximum range of values
    rnge = max(np.abs(x1-x0), np.abs(y1-y0))
    # Get line indices
    x = np.round(np.linspace(x0, x1, rnge+1)).astype(int)
    y = np.round(np.linspace(y0, y1, rnge+1)).astype(int)
    # Set the image at the line indices to the specified color
    output[y, x] = np.array(color)
    return output

plt.close('all')
view = viewer([img for img in mouse_tracking], title=[f'Mouse frame {i}' for i in range(mouse_tracking.shape[0])], widgets=True)

## 3.A. Mouse extraction
[Back to table of contents](#ToC_2_Morphology)

In the cell below, **for 1 point**, implement the function `extract_mouse`, which extracts **the body of the mouse** (without the tail) using **only one** morphological operator followed by a **threshold operation**. Use the input parameter `n` for the size of the chosen structuring element and the parameter `T` for the threshold value.
<div class='alert alert-info'>
    <b>Note:</b> The pixels corresponding to the mouse body should be set to <b>white (255)</b> and the background to <b>black (0)</b>.
</div>

In [ ]:
# Extracts the mouse using one morphological operater with a structuring element of size n
# followed by a thresholding operation using the threshold value T
def extract_mouse(img, n, T):
    output = np.zeros(img.shape)
    
    # YOUR CODE HERE
    
    return output

In the next cell we define a viewer with an extra widget for you to find suitable values for the two parameters `n` and `T`. Run the cell and play around with the two sliders until you find a combination that works for you.

In [ ]:
# Define widgets
n_slider = widgets.IntSlider(min=1, max=49, value=25, step=2, description='n')
T_slider = widgets.IntSlider(min=0, max=255, value=127, step=1, description='T')
button = widgets.Button(description='Extract mouse')

# Define callback functions
def identity_callback(img):
    return img
def callback_fun(img):
    n = n_slider.value
    T = T_slider.value
    return extract_mouse(img, n, T)

# Create image and title lists
mouse = [img for img in mouse_tracking]
extracted_titles = [f'Extracted mouse frame {i}' for i in range(mouse_tracking.shape[0])]
titles = [f'Mouse frame {i}' for i in range(mouse_tracking.shape[0])]

# Display
plt.close('all')
view = viewer([mouse, mouse], title=[titles, extracted_titles], widgets=True, subplots=(1,2), joint_zoom=True,
              callbacks=[identity_callback, callback_fun], new_widgets=[n_slider, T_slider, button])

In the cell below, assign the values for `n` and `T` that work for you.

In [ ]:
# Define the parameters n and T
n = None
T = None
# YOUR CODE HERE

In [ ]:
# Check that the given values make sense
if n % 2 == 0:
    print('WARNING:\nYour value for n should be odd.')
if T > 255 or T < 0:
    print('WARNING:\nYour value for T should be between 0 and 255.')

And now we'll perform a simple sanity check that tests if your image is binary with values $\{0, 255\}$.

In [ ]:
# Check that the output is binary
check_bin = extract_mouse(mouse_tracking[0], n, T)
if not (len(np.unique(check_bin)) == 2 and np.allclose(np.unique(check_bin), [0, 255])):
    print('WARNING!\nThe output is not binary with values {0, 255}. Your output consists of the following values:\n' + 
          str(np.unique(check_bin)))
else:
    print("Good, the output is binary with values {0, 255}.")

## 3.B. Implement the tracking algorithm
[Back to table of contents](#ToC_2_Morphology)

Now that we have a function which can extract the mouse from the images, we will use it to track the mouse's position over time. To estimate the position of the mouse, we will use the `cog` function that calculates the center of gravity of the extracted mouse. First, let's make sure that this position calculation works. Run the cell below, to apply the `cog` function to the output of your `extract_mouse` function and check that the mouse position is correctly displayed for each image of the sequence `mouse_tracking`.

In [ ]:
plt.close('all')
view = viewer([show_cog(extract_mouse(img, n, T)) for img in mouse_tracking], 
              title=[f'Extracted mouse cog for frame {i}' for i in range(mouse_tracking.shape[0])], widgets=True)

If you think that the position of the mouse is correctly tracked, proceed with the exercise, otherwise check your `extract_mouse` function again and try to improve it such that the tracking works better.

The next step is to implement the `track_mouse` function. This function combines all the previously declared functions to incrementally display the path of the mouse over the whole image sequence. In the cell below, **for 1 point**, complete the tracking algorithm in the function `track_mouse`, which should perform the following tasks for each image of the input sequence:

1. Extract the mouse using your `extract_mouse` function.
2. Calculate the center of gravity using the `cog` function.
3. Save the obtained location in the `pos` array for later use.
4. Draw a red cross at the current mouse position using the `show_cog` function.
5. Draw red lines representing the previous path of the mouse (from frame 0 up to the current frame).
6. Store the current output frame that has the current mouse location as well as the previous path drawn on it in the `out` array.

<div class='alert alert-info'>
    <b>Notes:</b><ul><li>
    Draw the mouse position and the previous path onto the original <code>mouse_tracking</code> frames, not on the extracted mouse image.</li><li>
    Use your values for <code>n</code> and <code>T</code> that you specified above for the <code>extract_mouse</code> function.</li></ul>
</div>

Below you can find a brief description of the input and output parameters of the three functions that we already defined for you:

`cog(img)`:
* `img` : The image of which the center of gravity should be calculated.

returns `(c_x, c_y)` : Tuple representing the x and y location of the center of gravity.
 
`show_cog(img, cog_pos=None, mask=None)`:
* `img` : The image on which the red cross is drawn. By default, this image is also used for the cog calculation.
* `cog_pos` : Tuple containing the center of gravity location $(c_x, c_y)$. If not defined, `img` is used to calculate the cog.
* `mask` : The image to use for the cog calculation. If not defined, `img` is used as the mask.

returns `img` with a red cross added at the center of gravity location.
 
`draw_line(img, color, x0, y0, x1, y1)`:
* `img` : The image on which the line should be drawn.
* `color` : The color of the line in the 8-bit RGB format `[R, G, B]`. Colors range from 0 to 255.
* `x0` : The x position of the starting point of the line.
* `y0` : The y position of the starting point of the line.
* `x1` : The x position of the ending point of the line.
* `y1` : The y position of the ending point of the line.

returns `img` with the line drawn on it.

In [ ]:
# Function that tracks a mouse over multiple frames and draws its path on the output frames
def track_mouse(img):
    assert img.ndim == 3, 'The input image should be a stack of images.'
    # The output will be a stack of rgb images of exactly the same shape as img
    output = np.zeros((img.shape[0], img.shape[1], img.shape[2], 3), dtype=np.uint8)
    # This array is used to store the positions of the mouse's center of gravity for tracking
    pos = np.zeros((img.shape[0], 2), dtype=int)
    
    # Go through every frame of the input stack
    for i in range(img.shape[0]):
        # Extract the current frame
        frame = img[i]
        
        # Extract the mouse mask
        # YOUR CODE HERE
        
        # Calculate the center of gravity
        # YOUR CODE HERE
        
        # Save mouse location for tracking in the pos array
        # YOUR CODE HERE

        # Add cross at the center of gravity (use the already calculated cog to improve performance!)
        # YOUR CODE HERE
        
        # For every frame except the first one, draw all the previous tracking lines in red
        red_color = [255, 0, 0]
        if i > 0:
            for j in range(1, i+1):
                # YOUR CODE HERE
        
        # Insert the current frame into the output array
        # YOUR CODE HERE
    
    return output

Run the next two cells to perform some very basic sanity checks that ensure the correct output shape of your function. They do not test if the output is correct though.

In [ ]:
# Check that there are the same amount of images in the output as in the input stack
if len(track_mouse(mouse_tracking)) != len(mouse_tracking):
    print('WARNING!:\nThere should be the same amount of images in the output as in the input of the function.')

In [ ]:
# Check that the output images are RGB
if track_mouse(mouse_tracking).shape[-1] != 3:
    print('WARNING!:\nThe output images should be in the RGB color format.')

Run the cell below to apply your `track_mouse` function to the image sequence `mouse_tracking`.

In [ ]:
# Track the mouse
plt.close('all')
view = viewer([img for img in track_mouse(mouse_tracking)], 
              title=[f'Mouse tracking frame {i}' for i in range(mouse_tracking.shape[0])], widgets=True)

# 4. Cartoonize your picture! (Optional)
[Back to table of contents](#ToC_2_Morphology)

Choose a natural picture of your choice from the internet or from your own collection (it shouldn't be too large, otherwise the operations will take a long time). If you're too busy to search for an image yourself you can also use the _natural_image.jpg_ provided in the `images` directory. Using a combination of morphological operators, arithmetic operators, inversion, and threshold operations, give a cartoon effect to your picture! You can try to make it look like a painting or to give it a distortion effect, explore the possibilities! 
<div class="alert alert-info">

<b>Note</b>: You can invert an image using <a href="https://docs.opencv.org/2.4/modules/core/doc/operations_on_arrays.html#bitwise-not"><code>cv.bitwise_not(img)</code></a>

</div>

In [ ]:
orig = cv.cvtColor(cv.imread('images/natural_image.jpg'), cv.COLOR_BGR2RGB)
img = orig

# YOUR CODE HERE

cartoon_viewer = viewer([orig, img], title=['Original', 'Cartoonized'], subplots=(1,2))

<div class="alert alert-success">
    Congratulations on finishing the Morphology lab!
</div>
    
Make sure to save your notebook (you might want to keep a copy on your personal computer) and upload it to <a href="https://moodle.epfl.ch/mod/assign/view.php?id=1123107">Moodle</a>, <b>in a zip file with the first part of this lab</b>.
</p>

* Please do not rename the notebook!
* Name the `zip` file *Morphology_Lab.zip*.